In [ ]:
import matplotlib.pyplot as plt
from src.models import *
from src.model_analysis.utils import same_output
from src.post_quant.cle import cle_for_resmlp
from src.post_quant.bias_absorb import ba_for_resmlp
from src.model_analysis.visualize import layer_dist, act_dist, simulate_input, scale_plot, add_value_labels

# org_model = resmlp_24(pretrained=False).cuda().eval()
model = resmlp_24(pretrained=False).eval()
model.load_state_dict(torch.load("ResMLP_S24_ReLU_fp32_80.602.pth"))

model_norm = resmlp_24_norm(pretrained=False).eval()
model_norm.load_state_dict(torch.load("qat_norm.pth")["model"])
# qmodel = q_resmlp(model)

# # test CLE
# org_model = resmlp_24(pretrained=False).eval()
# org_model.load_state_dict(torch.load("ResMLP_S24_ReLU_fp32_80.602.pth"))
# model = resmlp_24(pretrained=False).eval()
# model.load_state_dict(torch.load("ResMLP_S24_ReLU_fp32_80.602.pth"))
# cle_for_resmlp(model)
# print(same_output(model, org_model, eps=1e-5))

In [ ]:
qmodel_org = q_resmlp(model)
qmodel_org.load_state_dict(torch.load("resmlp_org.pth", map_location='cpu')['state_dict'], strict=False)
qmodel_org.blocks[0].add_1.mult

In [ ]:
for a in torch.load("resmlp_norm.pth", map_location='cpu')['state_dict']:
    if a.startswith("blocks.0.add_1.mult"):
        print(a)

In [ ]:
x = torch.load("resmlp_norm.pth", map_location='cpu')['state_dict']['blocks.0.norm1.scale']
torch.tensor([x])

In [ ]:
# ckpt = torch.load("resmlp_norm.pth", map_location='cpu')

# for i in range(0, 24):
#     for b in range(0,2):
#         ckpt['state_dict'][f'blocks.{i}.norm{b+1}.scale'] = torch.tensor([ckpt['state_dict'][f'blocks.{i}.norm{b+1}.scale']])

# torch.save(ckpt, "resmlp_norm.pth")

In [ ]:
qmodel_norm = q_resmlp_norm(model_norm)
qmodel_norm.load_state_dict(torch.load("resmlp_norm.pth", map_location='cpu')['state_dict'], strict=False)
qmodel_org.blocks[0].add_1.mult

In [ ]:
torch.round(torch.tensor([-12 / 2**3]))

In [ ]:
def orgDivRound(x, s):
    return torch.round(x / s + 0.000001)

def orgDivp5Floor(x, s):
    return torch.floor(x / s + 0.5)

def round(x):
    return torch.round(x + 0.000001)

def p5Floor(x):
    return torch.floor(x + 0.5)

def eDivRound(x, m, e):
    fix = torch.bitwise_left_shift(1, e-1)
    mul = x * m
    return torch.bitwise_right_shift(mul + fix, e)

In [ ]:
x = -15
print(x / qmodel_org.blocks[0].add_1.s)
print(orgDivRound(x, qmodel_org.blocks[0].add_1.s))
print(orgDivp5Floor(x, qmodel_org.blocks[0].add_1.s))
# print(eDivRound(x, qmodel_org.blocks[0].add_1.mult, qmodel_org.blocks[0].add_1.shift))

In [ ]:
x = torch.tensor([12])
print(x/ 2**3)
print(orgDivRound(x, 2**3))
print(orgDivp5Floor(x, 2**3))

In [ ]:
x = torch.tensor([1.5])
print(x)
print(round(x))
print(p5Floor(x))

In [ ]:
x = torch.tensor([-1.5])
print(torch.round(x))

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

# cle_for_resmlp(model)
layer_dist(model, 0, 23, name="ResMLP-org: Weight Distribution (after CLE)", ax=ax[0])
# cle_for_resmlp(model)
# res_
# cle_for_resmlp(model_norm)
layer_dist(model_norm, 0, 23, name="ResMLP-BN: Weight Distribution (before CLE)", ax=ax[1])

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

model.cuda()
model_norm.cuda()
act_dist(model, 0, 23, name='ResMLP-org: Activation Distribution of Each Layer', ax=ax[0], real_sim=True)
act_dist(model_norm, 0, 23, name='ResMLP-BN: Activation Distribution of Each Layer', ax=ax[1], real_sim=True)

In [ ]:
# model = resmlp_24(pretrained=True).eval()
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

# model.cuda()
# model_norm.cuda()
model.cpu()
model_norm.cpu()
labels, data = scale_plot(model, 0, 23, show_layers=["gamma_1", "gamma_2"], name='Scale of gamma_1, gamma_2', ax=ax[0])
for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    label.set_fontsize(16)
ax[0].plot(labels, data)
add_value_labels(ax[0], skip_cnt=1, precision=2)

labels, data = scale_plot(model_norm, 0, 23, show_layers=["gamma_1", "gamma_2"], name='Scale of gamma_1, gamma_2', ax=ax[1])
for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
    label.set_fontsize(16)
ax[1].plot(labels, data)
add_value_labels(ax[1], skip_cnt=1, precision=2)



In [ ]:
model = resmlp_24(pretrained=True).eval()
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

act_dist(model_norm, 0, 23, name='ResMLP-BN: Activation Distribution of Each Layer', ax=ax[0])

act_dist(model_norm, 0, 23, name='ResMLP-BN: Activation Distribution of Each Layer (without attn)', show_layers=["mlp.fc1", "mlp.fc2"], ax=ax[1])

In [ ]:
model = resmlp_24(pretrained=True).eval()
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

act_dist(model, 0, 23, name='Activation Distribution of Each Layer', ax=ax[0])

act_dist(model_norm, 0, 23, name='ResMLP-BN: Activation Distribution of Each Layer', ax=ax[1])

In [ ]:
# ckp = torch.load("./resmlp_norm2.pth", map_location='cpu')["model"]
# modified_dict = {}
# for k, v in ckp.items():
#     # if "norm" in k:
#     #     # print(k)
#     #     if ("weight" in k) or ("bias" in k):
#     #         continue
#     modified_dict[k] = v

# _ = model_norm.load_state_dict(ckp, strict=False)

checkpoint = torch.load("./resmlp_norm2.pth", map_location='cpu')["model"]
_ = model_norm.load_state_dict(checkpoint, strict=False)

In [ ]:
model_norm = resmlp_24_norm(pretrained=True).eval()

In [ ]:
import torchsummary as summary
summary.summary(model, (3, 224, 224), device="cuda")

In [ ]:
import torchsummary as summary
summary.summary(model_norm, (3, 224, 224), device="cuda")

In [ ]:
# getattr(model, f"layer0")
b_temp = None
for i in range(1):
    for n, m in model_norm.blocks[i].named_modules():
        print(n)
        if "norm1" in n:
            b_temp = m.cpu().eval()
            # n_parameters = sum(p.numel() for p in m.parameters() if p.requires_grad)
            # print(n_parameters)

In [ ]:
import numpy as np

In [ ]:
C = 2#196
L = 5#384

# x = np.array(np.random.rand(C, L), dtype=np.float32)
# x = torch.from_numpy(x).unsqueeze(0)


# x = torch.tensor([[[1., 2., 3., 4., 5.],
#          [10., 20., 30., 40., 50.]]])
x = torch.tensor([[[1., 2., 3., 4., 5.], [2., 4., 6., 8., 10.]], [[10., 20., 30., 40., 50.], [20., 30., 40., 50., 60.]]])
print(x.shape)
print(x)

In [ ]:
x.transpose(1,2)

In [ ]:
qqq = x.transpose(1,2)

In [ ]:
qqq - torch.mean(qqq, dim=(0, 2))

In [ ]:
# linear = nn.Linear(L, L, bias=True)
bn = nn.BatchNorm1d(5, momentum=1, affine=True)

# x = linear(x)
x = torch.tensor([[[1., 2., 3., 4., 5.], [2., 4., 6., 8., 10.]], [[10., 20., 30., 40., 50.], [20., 30., 40., 50., 60.]], [[1., 2., 3., 4., 5.], [2., 4., 6., 8., 10.]], [[10., 20., 30., 40., 50.], [20., 30., 40., 50., 60.]]])
# print(x.transpose(1,2).shape, '\n', x.transpose(1,2))

_ = bn(x.transpose(1,2)).transpose(1,2)

# # bn(x.transpose(1,2)).transpose(1,2)
# # bn(x.transpose(1,2)).transpose(1,2)
# print(bn.running_mean, bn.running_var)
# print(torch.mean(x, dim=(0, 1)), torch.var(x, dim=(0, 1)))

bn.eval()
print(bn(x.transpose(1,2)).transpose(1,2))

In [ ]:
bn.running_mean

In [ ]:
std = (bn.running_var + bn.eps).sqrt()
t = torch.diag(bn.weight / std)
b = bn.weight * bn.running_mean / std
(x @ t) - b + bn.bias

In [ ]:
std = (bn.running_var + bn.eps).sqrt()
b = bn.running_mean / std
x * std + b

In [ ]:
x.shape

In [ ]:
def _fuse_bn(self, conv_or_fc, bn):
    std = (bn.running_var + bn.eps).sqrt()
    t = bn.weight / std
    if conv_or_fc.weight.ndim == 4:
        t = t.reshape(-1, 1, 1, 1)
    else:
        t = t.reshape(-1, 1)
    return conv_or_fc.weight * t, bn.bias - bn.running_mean * bn.weight / std

In [ ]:
z = (x - bn.running_mean) * torch.rsqrt(bn.running_var + bn.eps)

#bn.weight * z + bn.bias
z

In [ ]:
x = torch.tensor([[[1., 2., 3., 4., 5.], [2., 4., 6., 8., 10.]], [[10., 20., 30., 40., 50.], [20., 30., 40., 50., 60.]]])
a = bn(x.transpose(1,2)).transpose(1,2)
a

In [ ]:
bn.running_mean

In [ ]:
def fuse_linear_bn_weights(linear_w, linear_b, bn_rm, bn_rv, bn_eps, bn_w, bn_b):
    if linear_b is None:
        linear_b = torch.zeros_like(bn_rm)
    bn_scale = bn_w * torch.rsqrt(bn_rv + bn_eps)

    fused_w = linear_w * bn_scale.unsqueeze(-1)
    fused_b = (linear_b - bn_rm) * bn_scale + bn_b

    return torch.nn.Parameter(fused_w), torch.nn.Parameter(fused_b)

In [ ]:
def fuse_1(linear, bn):
    w = linear.weight
    print(w.size())
    mean = bn.running_mean
    var = bn.running_var
    eps = bn.eps
    beta = bn.weight
    gamma = bn.bias
    if linear.bias is not None:
        b = linear.bias
    else:
        b = torch.zeros_like(mean.shape)

    
    bn_scale = beta * torch.rsqrt(var + eps)
    w = w * bn_scale#.unsqueeze(-1)
    b = (b - mean)*bn_scale + gamma
    fused_linear = nn.Linear(linear.in_features, linear.out_features)
                                             
    fused_linear.weight = nn.Parameter(w)
    fused_linear.bias = nn.Parameter(b)
    return fused_linear

In [ ]:
new_l = fuse_1(linear, bn)
new_l(x)

In [ ]:
class M(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        self.bn1d = nn.BatchNorm1d(1)

    def forward(self, x):
        x = self.linear(x)
        x = self.bn1d(x)
        return x

In [ ]:
from torch.ao.quantization.quantize_fx import (
    prepare_fx,
    convert_fx,
    prepare_qat_fx,
    fuse_fx,
)

m = M().eval()
m = fuse_fx(m)
m

In [ ]:
w, b = fuse_linear_bn_weights(linear.weight, linear.bias, bn.running_mean, bn.running_var, bn.eps, bn.weight, bn.bias)
F.linear(x, w, b)

In [ ]:
bn.running_mean
bn.running_var
bn.weight
bn.bias

In [ ]:
denom = torch.sqrt(bn.running_var + bn.eps)
b = bn.bias - bn.weight * bn.running_mean / denom
a = bn.weight / denom

In [ ]:
x

In [ ]:
F.linear(x, torch.eye(196).mul_(a), b)

In [ ]:
F.linear(x.transpose(1,2), torch.diag(a), b).transpose(1,2)

In [ ]:
mean = x.mean(dim=0).unsqueeze(dim=0)
var = ((x-mean)**2).mean(dim=0).unsqueeze(dim=0)
out = (x - mean / torch.sqrt(var + 1e-5))
out

In [ ]:
model = resmlp_24(pretrained=True).cuda().eval()
model_norm = resmlp_24_norm(pretrained=True).cuda().eval()
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

act_dist(model, 0, 23, name='ResMLP-org: Activation Distribution of Each Layer', ax=ax[0], real_sim=False)
act_dist(model_norm, 0, 23, name='ResMLP-BN: Activation Distribution of Each Layer', ax=ax[1], real_sim=False)

In [ ]:
x

In [ ]:
out

In [ ]:
bn.running_mean.shape

In [ ]:
(x - bn.running_mean) / torch.sqrt(bn.running_var + bn.eps)

In [ ]:
bn.weight * (x - bn.running_mean) / torch.sqrt(bn.running_var + bn.eps) + bn.bias

In [ ]:
output_factor = bn.weight / torch.sqrt(bn.running_var + bn.eps)
weight = torch.diag(output_factor)
bias = - output_factor * bn.running_mean + bn.bias

In [ ]:
F.linear(x, weight, bias)#.shape

In [ ]:
model_norm.blocks[0].gamma_2.weight.shape

In [ ]:
b_temp.weight

In [ ]:
running_std = torch.sqrt(b_temp.running_var + b_temp.eps)
scale_factor = b_temp.weight / running_std
scale_factor

In [ ]:
running_std = torch.sqrt(self.bn.running_var.detach() + self.bn.eps)
            scale_factor = self.bn.weight / running_std
            scaled_weight = self.conv.weight * scale_factor

In [ ]:
CHECKPOINT_PATH = "input8-conv-res16-act8.pth.tar"

model = resmlp_24()
qmodel = q_resmlp(model)
qmodel.load_state_dict(torch.load(CHECKPOINT_PATH)['state_dict'])

for n, m in qmodel.named_modules():
    if isinstance(m, QLinear):
        print(n)

In [ ]:
model = resmlp_24(pretrained=False).eval()
qmodel = q_resmlp(model)
qmodel.load_state_dict(torch.load(CHECKPOINT_PATH)['state_dict'])
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

label_1 = []
data_1 = []
for n, m in qmodel.blocks.named_modules():
    if "add_1.observer" in n:
        label_1.append(n)
        data_1.append([m.scale.item()])

ax[0].set_title('Rescale After add_1 (fp)', size=30)
ax[0].tick_params(labelrotation=30)
for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    label.set_fontsize(16)
ax[0].plot(label_1, data_1)
add_value_labels(ax[0], skip_cnt=2, precision=4)

label_2 = []
data_2 = []
for n, m in qmodel.named_modules():
    if "add_2.observer" in n:
        label_2.append(n)
        data_2.append(m.scale.item())

ax[1].set_title('Rescale After add_2 (fp)', size=30)
ax[1].tick_params(labelrotation=30)
for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
    label.set_fontsize(16)
ax[1].plot(label_2, data_2)
add_value_labels(ax[1], skip_cnt=2, precision=4)

In [ ]:
model = resmlp_24(pretrained=False).eval()
qmodel = q_resmlp(model)
qmodel.load_state_dict(torch.load(CHECKPOINT_PATH)['state_dict'])
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

label_1 = []
data_1 = []
for n, m in qmodel.blocks.named_modules():
    if "add" in n and "observer" not in n:
        label_1.append(n)
        scale = (m.res_mult[0].type(torch.double) / (2.0 ** m.res_shift[0]).type(torch.double)).type(torch.float)
        data_1.append(scale)

ax[0].set_title('Alignment for Residual Layer (int16 -> int32)', size=30)
ax[0].tick_params(labelrotation=30)
for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    label.set_fontsize(16)
ax[0].plot(label_1, data_1)
add_value_labels(ax[0], skip_cnt=1, precision=2)


label_2 = []
data_2 = []
for n, m in qmodel.named_modules():
    if "add" in n and "observer" not in n:
        label_2.append(n)
        scale = (m.mult[0].type(torch.double) / (2.0 ** m.shift[0]).type(torch.double)).type(torch.float)
        data_2.append(scale)

ax[1].set_title('Rescale After Residual Connection (int32 -> int8)', size=30)
ax[1].tick_params(labelrotation=30)
for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
    label.set_fontsize(16)
ax[1].plot(label_2, data_2)
add_value_labels(ax[1], skip_cnt=2, precision=2)

In [ ]:
model = resmlp_24(pretrained=False).eval()
qmodel = q_resmlp(model)
qmodel.load_state_dict(torch.load(CHECKPOINT_PATH)['state_dict'])
fig, ax = plt.subplots(2,1, figsize=(20, 10))
ax[0].get_xaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)

label_1 = []
data_1 = []
for n, m in qmodel.named_modules():
    if isinstance(m, QLinear):
        label_1.append(n)
        data_1.append(m.observer.scale.item())

ax[0].set_title('Scales for Weights Layers (fp -> int8)', size=30)
ax[0].tick_params(labelrotation=30)
for label in (ax[0].get_xticklabels() + ax[0].get_yticklabels()):
    label.set_fontsize(16)
ax[0].plot(label_1, data_1)
add_value_labels(ax[0], skip_cnt=9, precision=3)


# label_2 = []
# data_2 = []
# for n, m in qmodel.named_modules():
#     if "add" in n and "observer" not in n:
#         label_2.append(n)
#         scale = (m.mult[0].type(torch.double) / (2.0 ** m.shift[0]).type(torch.double)).type(torch.float)
#         data_2.append(scale)

# ax[1].set_title('Rescale After Residual Connection (int32 -> int8)', size=30)
# ax[1].tick_params(labelrotation=30)
# for label in (ax[1].get_xticklabels() + ax[1].get_yticklabels()):
#     label.set_fontsize(16)
# ax[1].plot(label_2, data_2)
# add_value_labels(ax[1], skip_cnt=2, precision=2)